# TF ML TRAINING (MNIST DATA - CONVOLUTIONAL NETWORK)

In [38]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


### Start TensorFlow InteractiveSession

In [39]:
import tensorflow as tf
sess = tf.InteractiveSession()

### Placeholders

In [40]:
# The shape argument to placeholder is optional, but it allows TensorFlow
# to automatically catch bugs stemming from inconsistent tensor shapes.
x = tf.placeholder(tf.float32, shape=[None, 784]) # input 28x28 image-pixel representation
y_ = tf.placeholder(tf.float32, shape=[None, 10]) # real "labels"

### Variables

In [41]:
W = tf.Variable(tf.zeros([784,10])) # softmax regression model weights
b = tf.Variable(tf.zeros([10])) # softmax regression model bias

In [42]:
sess.run(tf.initialize_all_variables())

### Predicted class and cost function

In [43]:
y = tf.nn.softmax(tf.matmul(x,W) + b) # matmul => math multiplication

In [44]:
# Our cost function will be the cross-entropy between the target and the model's prediction.
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

### Training the model

In [45]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [46]:
for i in range(1000):
    batch = mnist.train.next_batch(50)
    train_step.run(feed_dict={x: batch[0], y_: batch[1]})
# OR
    # batch_xs, batch_ys = mnist.train.next_batch(50)
    # train_step.run(feed_dict={x: batch_xs, y_: batch_ys})    

### Evaluate the model

In [47]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1)) # where are the max of both one-hot arrays?

In [48]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [49]:
print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9092


Getting 91%-92% accuracy on MNIST is bad. It's almost embarrassingly bad. In this section, we'll fix that, jumping from a very simple model to something moderately sophisticated: **a small convolutional neural network**. This will get us to around 99.2% accuracy -- not state of the art, but respectable.

# Build a Multilayer Convolutional Network

### Weight Initialization

In [50]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

### Convolution and Pooling



In [51]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

### First Convolutional Layer

We can now implement our first layer. It will consist of convolution, followed by max pooling. The convolutional will compute 32 features for each 5x5 patch. Its weight tensor will have a shape of [5, 5, 1, 32]. The first two dimensions are the patch size, the next is the number of input channels, and the last is the number of output channels. We will also have a bias vector with a component for each output channel.

In [52]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

To apply the layer, we first reshape **x** to a 4d tensor, with the second and third dimensions corresponding to image width and height, and the final dimension corresponding to the number of color channels.



In [53]:
x_image = tf.reshape(x, [-1,28,28,1])

We then convolve x_image with the weight tensor, add the bias, apply the ReLU function, and finally max pool.



In [54]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

### Second Convolutional Layer


In order to build a deep network, we stack several layers of this type. The second layer will have 64 features for each 5x5 patch.



In [55]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

### Densely Connected Layer

Now that the image size has been reduced to 7x7, we add a fully-connected layer with 1024 neurons to allow processing on the entire image. We reshape the tensor from the pooling layer into a batch of vectors, multiply by a weight matrix, add a bias, and apply a ReLU.



In [56]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

### Dropout

To reduce overfitting, we will apply dropout before the readout layer. We create a placeholder for the probability that a neuron's output is kept during dropout. This allows us to turn dropout on during training, and turn it off during testing. TensorFlow's tf.nn.dropout op automatically handles scaling neuron outputs in addition to masking them, so dropout just works without any additional scaling.1

In [57]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

### Readout Layer



Finally, we add a softmax layer, just like for the one layer softmax regression above.


In [58]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

### Train and Evaluate the Model

How well does this model do? To train and evaluate it we will use code that is nearly identical to that for the simple one layer SoftMax network above. The differences are that: we will replace the steepest gradient descent optimizer with the more sophisticated ADAM optimizer; we will include the additional parameter keep_prob in feed_dict to control the dropout rate; and we will add logging to every 100th iteration in the training process.


In [59]:
# Cross Entropy Cost Function
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))

# Training 
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

# Checking predictions (argmax)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))

# Accuracy
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Init vars
sess.run(tf.initialize_all_variables())

for i in range(20000):
    batch = mnist.train.next_batch(50)
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x:batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.18
step 100, training accuracy 0.8
step 200, training accuracy 0.84
step 300, training accuracy 0.84
step 400, training accuracy 0.96
step 500, training accuracy 1
step 600, training accuracy 0.96
step 700, training accuracy 1
step 800, training accuracy 1
step 900, training accuracy 0.96
step 1000, training accuracy 0.94
step 1100, training accuracy 0.96
step 1200, training accuracy 1
step 1300, training accuracy 1


Exception AssertionError: AssertionError("Nesting violated for default stack of <type 'weakref'> objects",) in <bound method InteractiveSession.__del__ of <tensorflow.python.client.session.InteractiveSession object at 0x1175cc450>> ignored


KeyboardInterrupt: 